# 1) Descargar la Base de Datos

In [ ]:
!gdown 1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX

Downloading...
From: https://drive.google.com/uc?id=1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX
To: /content/data_berka.db
100% 70.2M/70.2M [00:00<00:00, 152MB/s]


1.1) Cargamos la extensión sql (nos permite ejecutar comandos sql directamente en colab)

In [ ]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


1.2) Conectar a la Base de Datos

In [ ]:
%sql sqlite:///data_berka.db

1.3) Comprobamos el esquema de la BBDD
Para ejecutar comandos sql, se debe añadir:
 "%sql comando"
 o en múltiples líneas:
"%%sql
comando_línea1
comando_línea2"

In [ ]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type = "table"

 * sqlite:///data_berka.db
Done.


name,type
DISTRICT,table
ACCOUNT,table
TRANS,table
CLIENT,table
DISPOSITION,table
CREDIT_CARD,table
LOAN,table
PERMANENT_ORDER,table


# 2) Ejercicios

2.1) Muestre los datos de la tabla cliente

In [ ]:
%sql SELECT * FROM CLIENT LIMIT 5

 * sqlite:///data_berka.db
Done.


client_id,birth_number,district_id
1,706213,18
2,450204,1
3,406009,1
4,561201,5
5,605703,5


2.2) Liste todos los clientes con cuentas activas y préstamos, incluyendo dinero prestado y saldos de cuenta

In [ ]:
%%sql SELECT
  c.client_id AS Cliente,
  ROUND(SUM(l.amount), 2) AS Importe_Prestado,
  ROUND(SUM(CASE
      WHEN t.type = 'PRIJEM' THEN t.amount
      WHEN t.type = 'VYDAJ' THEN -t.amount
      ELSE 0
    END), 2) AS Balance_Cuenta
FROM CLIENT c
LEFT JOIN DISPOSITION d ON d.client_id = c.client_id
LEFT JOIN ACCOUNT a ON a.account_id = d.account_id
LEFT JOIN LOAN l ON l.account_id = a.account_id
LEFT JOIN TRANS t ON t.account_id = a.account_id
WHERE d.type = 'OWNER'
GROUP BY c.client_id
LIMIT 5

 * sqlite:///data_berka.db
Done.


Cliente,Importe_Prestado,Balance_Cuenta
1,38695056.0,273536.2
4,None,51096.2
6,None,33720.6
7,None,28088.6
8,None,47667.9


2.3) Identifique clientes con múltiples cuentas.

In [ ]:
%%sql
SELECT
  c.client_id AS Cliente,
  COUNT(a.account_id) AS Total_Cuentas
FROM client c
LEFT JOIN disposition d ON d.client_id = c.client_id
LEFT JOIN account a ON a.account_id = d.account_id
GROUP BY c.client_id
HAVING Total_Cuentas > 1

 * sqlite:///data_berka.db
Done.


Cliente,Total_Cuentas
1,2


In [ ]:
%%sql UPDATE disposition set client_id = 1 WHERE disp_id = 2

 * sqlite:///data_berka.db
1 rows affected.


[]

Revertimos los cambios.

In [ ]:
%%sql UPDATE disposition set client_id = 2 WHERE disp_id = 2

 * sqlite:///data_berka.db
(sqlite3.OperationalError) attempt to write a readonly database
[SQL: UPDATE disposition set client_id = 2 WHERE disp_id = 2]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


2.4) Recupere transacciones del año 1998, ordenadas por cantidad de transacción, e identifique las 10 transacciones más grandes.

In [ ]:
%%sql
SELECT
  trans_id AS ID,
  date AS Fecha,
  ROUND(amount, 2) AS Importe
FROM trans
WHERE date LIKE '98%'
ORDER BY amount DESC
LIMIT 10

 * sqlite:///data_berka.db
Done.


ID,Fecha,Importe
291346,981215,87400.0
1306770,980606,87300.0
1207200,980619,85800.0
1170721,980615,81900.0
706371,980610,80100.0
141211,980625,78300.0
1155373,980618,78000.0
623347,980612,77400.0
768915,980620,77100.0
816974,980621,76800.0


In [ ]:
%%sql
SELECT
  trans_id AS ID,
  date AS Fecha,
  ROUND(amount, 2) AS Importe
FROM trans
WHERE SUBSTR(date, 0, 3) = '98'
ORDER BY amount DESC
LIMIT 10

 * sqlite:///data_berka.db
Done.


ID,Fecha,Importe
291346,981215,87400.0
1306770,980606,87300.0
1207200,980619,85800.0
1170721,980615,81900.0
706371,980610,80100.0
141211,980625,78300.0
1155373,980618,78000.0
623347,980612,77400.0
768915,980620,77100.0
816974,980621,76800.0


2.5) ¿Cuál es el saldo promedio de cuentas con préstamos en comparación con aquellas sin préstamos?.

In [ ]:
%%sql
SELECT ROUND(AVG(t.balance), 2) AS Saldo_Promedio_SP
FROM account a
LEFT JOIN trans t ON t.account_id = a.account_id
LEFT JOIN loan l ON l.account_id = a.account_id
WHERE l.account_id IS NULL

 * sqlite:///data_berka.db
Done.


Saldo_Promedio_SP
36979.08


In [ ]:
%%sql
SELECT ROUND(AVG(t.balance), 2) AS Saldo_Promedio_CP
FROM account a
LEFT JOIN trans t ON t.account_id = a.account_id
LEFT JOIN loan l ON l.account_id = a.account_id
WHERE l.account_id IS NOT NULL

 * sqlite:///data_berka.db
Done.


Saldo_Promedio_CP
45467.14


In [ ]:
%%sql
SELECT
(SELECT ROUND(AVG(t.balance), 2)
FROM account a
LEFT JOIN trans t ON t.account_id = a.account_id
LEFT JOIN loan l ON l.account_id = a.account_id
WHERE l.account_id IS NULL) AS Saldo_Promedio_SP,
(SELECT ROUND(AVG(t.balance), 2)
FROM account a
LEFT JOIN trans t ON t.account_id = a.account_id
LEFT JOIN loan l ON l.account_id = a.account_id
WHERE l.account_id IS NOT NULL) AS Saldo_Promedio_CP


 * sqlite:///data_berka.db
Done.


Saldo_Promedio_SP,Saldo_Promedio_CP
36979.08,45467.14


2.6) Identifique clientes que hayan realizado más de 10 transacciones en el año 1998 específico pero que no hayan solicitado un préstamo.

In [75]:
%%sql
SELECT c.client_id AS Cliente
FROM client c
LEFT JOIN disposition d ON d.client_id = c.client_id
LEFT JOIN account a ON a.account_id = d.account_id
LEFT JOIN loan l ON l.account_id = a.account_id
LEFT JOIN trans t ON t.account_id = a.account_id
WHERE SUBSTR(t.date, 0, 3) = '98' AND l.account_id IS NULL
GROUP BY c.client_id
HAVING COUNT(t.trans_id) > 10
LIMIT 10


 * sqlite:///data_berka.db
Done.


Cliente
1
4
5
6
7
8
9
10
11
12


2.7) Segmentar a los clientes en función al saldo de su cuenta (por ejemplo, bajo, medio, alto) y analizar la distribución de solicitudes de préstamos en estos segmentos.

Definimos los límites de balance mínimos y máximos

In [76]:
%%sql
SELECT
  ROUND(MIN(balance),2) AS Min_Balance,
  ROUND(MAX(balance),2) AS Max_Balance
FROM trans;

 * sqlite:///data_berka.db
Done.


Min_Balance,Max_Balance
-41125.7,209637.0


Realizamos la consulta individualizada
Indicando rangos:
> BAJO: de Mín a 0
> MEDIO: de 0 a MAX/2
> ALTO: de MAX/2 a MAX

In [78]:
%%sql
WITH Limits AS (SELECT MAX(balance)/2 AS medium_limit FROM trans)
SELECT
  c.client_id,
  t.trans_id,
  t.balance AS last_balance,
  CASE
    WHEN MAX(t.balance) > 0 THEN 'BAJO'
    WHEN MAX(t.balance) > 0 AND MAX(t.balance) >= (SELECT medium_Limit FROM Limits) THEN 'MEDIO'
    WHEN MAX(t.balance) > (SELECT medium_Limit FROM Limits) THEN 'ALTO'
    ELSE 'NAN'
  END AS Calidad_Cliente
FROM client c
JOIN disposition d ON d.client_id = c.client_id
JOIN trans t ON t.account_id = d.account_id
JOIN (
  SELECT account_id, MAX(trans_id) AS max_trans_id
  FROM trans
  GROUP BY account_id
) latest_trans ON t.account_id = latest_trans.account_id AND t.trans_id = latest_trans.max_trans_id
GROUP BY c.client_id
LIMIT 10

 * sqlite:///data_berka.db
Done.


client_id,trans_id,last_balance,Calidad_Cliente
1,3530552,42628.1,BAJO
3,3530552,42628.1,BAJO
4,3530570,51096.1,BAJO
5,3530570,51096.1,BAJO
6,3530600,33720.7,BAJO
7,3530615,28088.3,BAJO
8,3530662,47668.0,BAJO
9,3530687,68495.5,BAJO
10,3530726,72617.2,BAJO
11,3530726,72617.2,BAJO


Obtenemos el resumen de estado final

In [87]:
%%sql
WITH Limits AS (
    SELECT MAX(TRANS.balance) * 0.5 AS medium_limit FROM TRANS
),
latest_trans AS (
    SELECT account_id, MAX(trans_id) AS max_trans_id
    FROM TRANS
    GROUP BY account_id
),
client_balances AS (
    SELECT
        CLIENT.client_id,
        TRANS.account_id,
        TRANS.balance AS LAST_BALANCE,
        CASE
            WHEN TRANS.balance <= 0 THEN 'BAJO'
            WHEN TRANS.balance > 0 AND TRANS.balance <= (SELECT medium_limit FROM Limits) THEN 'MEDIO'
            WHEN TRANS.balance > (SELECT medium_limit FROM Limits) THEN 'ALTO'
            ELSE 'NAN'
        END AS CALIDAD_CLIENTE,
        LOAN.amount AS importe_prestado
    FROM CLIENT
    JOIN DISPOSITION ON CLIENT.client_id = DISPOSITION.client_id
    JOIN TRANS ON DISPOSITION.account_id = TRANS.account_id
    JOIN LOAN ON TRANS.account_id = LOAN.account_id
    JOIN latest_trans ON TRANS.account_id = latest_trans.account_id AND TRANS.trans_id = latest_trans.max_trans_id
)
SELECT
    CALIDAD_CLIENTE,
    COUNT(DISTINCT client_id) AS num_clientes,
    SUM(importe_prestado) AS total_importe_prestado,
    ROUND(COUNT(DISTINCT client_id) * 100.0 / SUM(COUNT(DISTINCT client_id)) OVER (), 2) AS proporcion_clientes,
    ROUND(SUM(importe_prestado) * 100.0 / SUM(SUM(importe_prestado)) OVER (), 2) AS proporcion_importe_prestado
FROM client_balances
GROUP BY CALIDAD_CLIENTE
ORDER BY CALIDAD_CLIENTE;

 * sqlite:///data_berka.db
Done.


CALIDAD_CLIENTE,num_clientes,total_importe_prestado,proporcion_clientes,proporcion_importe_prestado
ALTO,29,3709860,3.51,2.96
BAJO,9,2079120,1.09,1.66
MEDIO,789,119750892,95.41,95.39
